In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import nltk
import random
import pickle
import re
from glob import glob
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Input, Dropout, LSTM, Embedding, Activation, Concatenate

In [ ]:
! pip install transformers
! pip install --upgrade accelerate
! pip install datasets
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1

In [ ]:
from datasets import load_dataset, list_datasets
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **From Image to Fill in the blank paragraph generation**

In [ ]:
with open("/content/drive/MyDrive/Dataset/images_features/images_features.pkl","rb") as f:
  images_features=pickle.load(f)

with open("/content/drive/MyDrive/Dataset/captions_dict_train.pkl", "rb") as file:
  captions_dict_train = pickle.load(file)

with open("/content/drive/MyDrive/Dataset/captions_dict_test.pkl", "rb") as file:
  captions_dict_test = pickle.load(file)

with open("/content/drive/MyDrive/Dataset/captions_dict_val.pkl", "rb") as file:
  captions_dict_val = pickle.load(file)

captions_dict_train_words = captions_dict_train.copy()

def preprocessed(txt):
  modified = txt.lower().replace('.',' .').replace("'","")
  modified = 'startofseq ' + modified + ' endofseq'
  return modified

count_words = {}


for k,v in captions_dict_train.items():
    captions_dict_train[k]=preprocessed(v)

#creating count_words dict with word and frequency
for k,v in captions_dict_train.items():
    for word in v.split():
        if word not in count_words:
            count_words[word] = 1
        else:
            count_words[word] += 1

#creating words_dict as vocabulary where only words occuring more than threshold is considered
THRESH = 5
count = 1
words_dict = {}
for k,v in count_words.items():
    if count_words[k] > THRESH:
        words_dict[k] = count
        count += 1

#captions_dict converting words into numbers from words_dict
for k, v in captions_dict_train.items():
    encoded = []
    for word in v.split():
        if word in words_dict.keys():
            encoded.append(words_dict[word])

    captions_dict_train[k] = encoded


N=32 #BatchSize
VOCAB_SIZE = len(words_dict)+1

def progressive_generator(photo_dict, caption_dict, MAX_LEN,VOCAB_SIZE):
  while 1:
      for i in range(0,len(caption_dict),N):
          caption = dict(list(caption_dict.items())[0+i: N+i])
          X, y_in, y_out = create_sequences(photo_dict,caption,MAX_LEN, VOCAB_SIZE)
          yield [X, y_in], y_out


def create_sequences(photo, caption, MAX_LEN,VOCAB_SIZE):
  #n_samples = 0
  X,y_in,y_out = [],[],[]

  for k, v in caption.items():
      for i in range(1, len(v)):
          X.append(photo[k])

          in_seq= [v[:i]]
          out_seq = v[i]

          in_seq = pad_sequences(in_seq, maxlen=MAX_LEN, padding='post', truncating='post')[0]
          out_seq = to_categorical([out_seq], num_classes=VOCAB_SIZE)[0]

          y_in.append(in_seq)
          y_out.append(out_seq)

  return np.array(X), np.array(y_in), np.array(y_out)

model150 = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/model150/image_caption_model_neewarch150.h5')

vocab_size = len(words_dict)+1
MAX_LEN = 0

for k, v in captions_dict_train.items():
  if len(v) > MAX_LEN:
    MAX_LEN = len(v)

inv_dict = {v:k for k, v in words_dict.items()}

In [ ]:
output_dir = "/content/drive/MyDrive/Dataset/gpt-neo/FITB_GPT_Neo_finetuned_fitb_epoch30"

model_name = "EleutherAI/gpt-neo-125M"
model = GPTNeoForCausalLM.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
def generate_caption(image_name):

  test_feature = images_features[image_name]
  test_img_path = '/content/drive/MyDrive/Dataset/stanford_images/'+image_name
  test_img = cv2.imread(test_img_path)
  test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)


  text_inp = ['startofseq']
  count = 0
  caption = ''
  while count < MAX_LEN:
    count += 1
    encoded = []
    for i in text_inp:
        encoded.append(words_dict[i])
    encoded = [encoded]
    encoded = pad_sequences(encoded, padding='post', truncating='post', maxlen=MAX_LEN)
    data_list=[np.array(test_feature).reshape(1,-1), np.array(encoded).reshape(1,-1)]
    prediction = np.argmax(model150.predict(data_list))
    sampled_word = inv_dict[prediction]
    caption = caption + ' ' + sampled_word

    if sampled_word == 'endofseq':
        break
    text_inp.append(sampled_word)

  caption= caption.replace('endofseq','')
  predicted= caption.split()
  actual= captions_dict_train_words[image_name].split()


  def convert_to_fill_in_the_blank(paragraph):

  # Tokenize the paragraph
    input_ids = tokenizer.encode(paragraph, add_special_tokens=False, return_tensors='pt')

    # Generate qa
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, do_sample=True)

    generated_fillqa = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_fillqa

  def remove_repeated_sentences(sentences):
    unique_sentences = set()
    for sentence in sentences:
        unique_sentences.add(sentence)
    return list(unique_sentences)

  # Split the paragraph into sentences using regex
  sentences = re.split(r'\.\s*', caption)

  # Add a space at the end of each sentence
  sentences_with_space = [sentence + " " for sentence in sentences if sentence]

  cleaned_para = remove_repeated_sentences(sentences_with_space)

  sentences_with_context = ["<CONTEXT_START> {} <CONTEXT_END>".format(sentence.strip()) for sentence in cleaned_para]

  output = []

  for sentence in sentences_with_context:
    converted_sentence = convert_to_fill_in_the_blank(sentence)
    output.append(converted_sentence)


  contexts = []
  questions = []
  answers = []

  for pattern in output:
    # Removing unwanted sentences
    pattern = re.sub(r'\d+,".*', '', pattern)

    # Extracting the desired parts
    context_match = re.search(r'<CONTEXT_START>(.*?)<CONTEXT_END>', pattern)
    question_match = re.search(r'<QUESTION_START>(.*?)<QUESTION_END>', pattern)
    answer_match = re.search(r'<ANSWER_START>(.*?)<ANSWER_END>', pattern)

    # Checking if answer pattern is found
    if answer_match:
        answer = answer_match.group(1)
    else:
        pass
    if context_match:
        context = context_match.group(1)
    else:
        pass
    if question_match:
        question = question_match.group(1)
    else:
        pass

    # Appending to relevant lists
    contexts.append(context.strip())
    questions.append(question.strip())
    answers.append(answer.strip())

  # plt.figure()
  # plt.imshow(test_img)
  # plt.title(image_name)
  # plt.show()

  # Printing
  # Create a new shuffled list
  shuffled_answers = random.sample(answers, len(answers))
  # Printing the 30 epochs
  # print("Contexts:", contexts)
  # print("Questions:", questions)
  # print("Answers:", answers)
  # print("Shuffled Answers:", shuffled_answers)

  return questions, answers

In [ ]:
import gradio as gr

# img_name='2395962.jpg'
image_name = gr.inputs.Textbox(placeholder="Enter image name ...")
output = gr.outputs.HTML(label="Fill in the blank Questions")

def process_image(image_name) :
  img_dir = "/content/drive/MyDrive/Dataset/stanford_images/"
  output = "<h2> Generated Fill in the blank question </h2>"
  question, answer = generate_caption(image_name)
  print("question : ", question)
  print("answer : ", answer)
  img = cv2.imread(img_dir + image_name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  output = output + "<b style='color:red;'> Answers: </b> <br> <p>"
  for ans in answer:
    output = output + ans + " | "
  output = output + "</p> <br> <br>"

  output = output + "<b style='color:yellow;'> Fill in the blank question: </b> <br> <p>"
  for ques in question:
    output = output + ques + ". "
  output = output + "</p>"

  return img, output

iface = gr.Interface(
  process_image,
  image_name,
  ["image", output])

iface.launch(debug = True, share = True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://daa351e03f31ad4d64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 47ms/step


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

question :  ['the ________ is green with white painted lines on it', 'a woman is ________ tennis', 'part of a wall can be seen behind the ________', 'she is holding a ________ in her hand', 'she is wearing a white tank top and ________', 'the building is both a front and a ________ with a few men sitting on the balcony.']
answer :  ['court', 'playing', 'woman', 'racket', 'black', 'men']
1/1 [==============================] - 0s 43ms/step


...

question :  ['the ________ is green with white painted lines on it', 'a woman is playing tennis', 'part of a wall can be seen behind the ________', 'she is holding a ________ in her hand', 'she is wearing a white tank top and ________', 'the building is all on the left and ________ are showing off the behind of the woman']
answer :  ['court', 'woman', 'woman', 'racket', 'black', 'woman']
1/1 [==============================] - 0s 43ms/step


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

question :  ['the ________ is green with white painted lines on it', 'a woman is ________ tennis', 'part of a wall can be seen behind the ________', 'she is ________ a racket in her hand', 'she is wearing a white tank top and ________', 'the background is tall and white, and the words are on the redwood of a front- ________']
answer :  ['court', 'playing', 'woman', 'holding', 'black', 'building']
1/1 [==============================] - 0s 79ms/step


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

question :  ['the ________ is green with white painted lines on it', 'a woman is playing ________', 'part of a wall can be seen behind the ________', 'she is ________ a racket in her hand', 'she is wearing a white tank top and ________', '________ of a building can be seen behind the woman']
answer :  ['court', 'tennis', 'woman', 'holding', 'black', 'part']
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://daa351e03f31ad4d64.gradio.live




---



---

